In [ ]:
from model_training.training import simple_conv_neural_network,conv_neural_network,save_model
from model_training.model_eval import compute_avg_mae
from src.config import CONFIG
import pandas as pd
import matplotlib.pyplot as plt

train_df = pd.read_csv("data/train_data.csv")
val_df = pd.read_csv("data/val_data.csv")
test_df = pd.read_csv("data/test_data.csv")

l2_penalty = CONFIG["l2_penalty"]
conv1_filters = CONFIG["conv1_filters"]
conv2_filters = CONFIG["conv2_filters"]
conv3_filters = CONFIG["conv3_filters"]
dense1_units = CONFIG["dense1_units"]
dense2_units = CONFIG["dense2_units"]
dropout_rate = CONFIG["dropout_rate"]
learning_rate = CONFIG["learning_rate"]
earlystop_patience = CONFIG["earlystop_patience"]
training_epochs = CONFIG["training_epochs"]

In [ ]:
model_1, history_1 = simple_conv_neural_network(
                                     conv1_filters,
                                     conv2_filters,
                                     conv3_filters,
                                     dense1_units,
                                     dense2_units,
                                     learning_rate,
                                     training_epochs)

plt.plot(history_1.history['denorm_mae'], label='Train MAE (denorm)')
plt.plot(history_1.history['val_denorm_mae'], label='Val MAE (denorm)')
plt.xlabel('Epoch')
plt.ylabel('MAE')
plt.legend()
plt.show()

train_mae = compute_avg_mae(train_df, model_1)
val_mae = compute_avg_mae(train_df, model_1)

print(f"Train mae: {train_mae}")
print(f"Val mae: {train_mae}")

save_model(model_1, "model_1")